<a href="https://colab.research.google.com/github/natatsypora/plotly_charts/blob/main/Bar_chart_plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd

## Get and read Data

In [ ]:
# get the file from Google Drive
! gdown -- 1U9GNdGpxm0rEo7WmE3Y9dKoDVlr7n5ye

In [ ]:
# read data
df_vs_last = pd.read_excel('/content/teunot_by_month_type2022.xlsx',
                           sheet_name='yoy', index_col=0, usecols=[0,1,5])
df_vs_last

,Total_2022,Total_2021
Month,,
January,881,862
February,826,916
March,957,1093
April,927,1046
May,985,935
June,892,1077
July,915,978
August,845,982
September,824,848


In [ ]:
# add a column with change vs previous year
df_vs_last['change_py'] = (df_vs_last['Total_2022'] - df_vs_last['Total_2021'])/df_vs_last['Total_2022']*100
df_vs_last.head(1)

,Total_2022,Total_2021,change_py
Month,,,
January,881,862,2.15664


##Create Bar Chart with annotations and columns overlay (bargap=0.5)

In [ ]:
# data for bar chart
x = df_vs_last.index.values
y1 = df_vs_last['Total_2021']
y2 = df_vs_last['Total_2022']
change = df_vs_last['change_py']

# define a bar width
bar_width = 0.4

# create the figure
fig = go.Figure()

# add the first bar
fig.add_trace(go.Bar(x=x, y=y1,
                     name='2021',
                     width=bar_width,
                     marker_color='rgba(31, 119, 180, 0.1)',
                     marker_line_color='rgb(8, 48, 107)',
                     marker_line_width=2)
             )

# add the second bar
fig.add_trace(go.Bar(x=x, y=y2,
                     name='2022',
                     width=bar_width,
                     marker_color='rgba(31, 119, 180, 1)',
                     marker_line_color='rgb(8, 48, 107)',
                     marker_line_width=2)
             )

# add annotations
for x, y, t in zip(x, y2, change):
    if t > 0:
        fig.add_annotation(x=x, y=y,
                           text=f'<b>{t:.0f}%</b>', xanchor ='right', yanchor='top',
                           font_size=12, font_color='green',
                           arrowhead=2, arrowcolor='green', ax=0,
                           borderpad=7, xshift=10, yshift=10,
                           arrowwidth=2, arrowside='start')
    else:
        fig.add_annotation(x=x, y=y,
                           text=f'<b>{t:.0f}%</b>', yanchor='top',
                           font_size=12, font_color='red',
                           arrowhead=2, arrowcolor='red', ax=0,
                           borderpad=5, xshift=25, yshift=10,
                           arrowwidth=2, arrowside='end')

# define layout and legend properties
fig.update_layout(title='Road Accidents 2022 vs 2021',
                  title_x=0.5, titlefont_size=20, font_color='rgb(8, 48, 107)',
                  barmode='group', bargap=0.5,  # move columns by 50 % relative to each other
                  margin=dict(t=80, b=10, l=10, r=10),
                  width=1000, height=400, template='simple_white',
                  legend=dict(orientation='h',
                              yanchor='bottom', y=1,
                              xanchor='right', x=0.95 ))

fig.show()

## Butterfly Chart

In [ ]:
# Create a simple dataframe
ages = pd.DataFrame(
    {"Age": ["0-19", "20-29", "30-39", "40-49", "50-59", ">=60"],
     "Male": [800, 2000, 4200, 5000, 2100, 800],
     "Female": [1000, 3000, 3500, 3800, 3600, 700],
    }
)

In [ ]:
def butterfly(df: pd.DataFrame, x1: str, x2: str, y: str,
              title: str, width=800, height=350):
    # create the figure
    fig = go.Figure()

    # add the first plot
    fig.add_trace(
        go.Bar(
            x=df[x1],
            y=df[y],
            orientation="h",
            name=x1,
            xaxis='x')
        )
    # add text to the first plot
    fig.add_trace(
        go.Scatter(
            showlegend=False, hoverinfo='skip',
            x=[0]*len(df[x1]),
            y=df[y],
            mode='text', text=df[x1], texttemplate='%{text:,.0f}'+' ',
            textfont_color='white', textposition='middle left',
            xaxis='x')
        )
    # add the second plot
    fig.add_trace(
        go.Bar(
            x=df[x2],
            y=df[y],
            orientation="h",
            name=x2,
            xaxis='x2', yaxis='y2')
        )
    # add text to the second plot
    fig.add_trace(
        go.Scatter(
            showlegend=False, hoverinfo='skip',
            x=[0]*len(df[x2]),
            y=df[y],
            mode='text', text=df[x2], texttemplate=' '+'%{text:,.0f}',
            textposition='middle right', textfont_color='white',
            xaxis='x2', yaxis='y2')
        )
    # define the layout properties
    fig.update_layout(
        title=title, title_x=0.5,
        font_size=14, paper_bgcolor='#E5ECF6',
        margin=dict(t=80, b=20, l=10, r=50),
        width=width, height=height,
        xaxis=dict(dict(domain=[0.0, 0.5],       # define the position for the first plot
                   range=[max(df[x1])*1.1, 0]),  # reverse xaxis for the first plot
                   anchor='x'),
        xaxis2=dict(dict(domain=[0.5, 1.0],
                         range=[0, max(df[x2])*1.1],
                         anchor='x2')),
        yaxis2=dict(anchor='y2', visible=False),
        # define the legend properties
        legend=(dict(orientation='h',
                     entrywidth=100,    #set the width of horizontal legend entries
                     x=0.6, y=1.10,
                     xanchor='center'))
                      )
    fig.update_xaxes(visible=False)
    fig.update_yaxes(title='Age Group', autorange='reversed',
                     linecolor='#ffffff', tickfont_size=14, tickfont_weight='bold',
                     ticklen=0, ticklabelstandoff=10)

    return fig

In [ ]:
butterfly = butterfly(df=ages, x1='Male', x2='Female', y='Age',
                      title="Age and Gender Butterfly Chart",)
butterfly.show()